In [187]:
import os
import pickle
import codecs
import nltk
from gensim.models import Word2Vec
from sklearn.metrics import f1_score,accuracy_score


In [2]:
from gensim.models import FastText

In [3]:
train_data=pickle.load(open(r"D:\PythonCodes\Sentiment-Analysis\Code\train_data_initial.dat","rb"))

In [4]:
#pickle.dump(train_data,open(r"D:\PythonCodes\Sentiment-Analysis\Code\train_data_initial_1.dat","wb"),protocol=2)

In [4]:
train_data.head()

,aspect,sentence,sentiment,snippet
0,Rumors,ab inbrev latest bid said unlikely win COMPANY...,0.127,latest bid said unlikely to win sabmiller's ap...
1,Appointment,COMPANY shakes board two new business chiefs t...,-0.074,"shakes up board with two new business chiefs, ..."
2,Appointment,press serco set appoint roy gardner ex COMPANY...,0.195,"set to appoint roy gardner, ex-centrica, as ch..."
3,Appointment,press COMPANY set appoint roy gardner ex centr...,0.122,"set to appoint roy gardner, ex-centrica, as ch..."
4,Strategy,barclays sells benchmark indices unit COMPANY,0.281,sells benchmark indices unit


In [5]:
len(train_data)

1173

In [6]:
train_data.head()

,aspect,sentence,sentiment,snippet
0,Rumors,ab inbrev latest bid said unlikely win COMPANY...,0.127,latest bid said unlikely to win sabmiller's ap...
1,Appointment,COMPANY shakes board two new business chiefs t...,-0.074,"shakes up board with two new business chiefs, ..."
2,Appointment,press serco set appoint roy gardner ex COMPANY...,0.195,"set to appoint roy gardner, ex-centrica, as ch..."
3,Appointment,press COMPANY set appoint roy gardner ex centr...,0.122,"set to appoint roy gardner, ex-centrica, as ch..."
4,Strategy,barclays sells benchmark indices unit COMPANY,0.281,sells benchmark indices unit


In [7]:
head_data=pickle.load(open('test_head.dat',"rb"))
post_data=pickle.load(open('test_post.dat',"rb"))

In [8]:
head_data.head()

,aspect,sentence,sentiment,snippets
0,Financial,COMPANY delivers small profit increase 2014,0.247,delivers small profit increase for
1,Financial,COMPANY reports million loss first quarter,-0.741,reports $583 million loss in first quarter
2,Sales,osborne extends COMPANY sell plan,-0.311,sell-off plan
3,Price Action,companies COMPANY shares hit 2016 revenue fore...,-0.375,hit as 2016 revenue forecast to fall
4,Strategy,COMPANY ends 27 year sponsorship tate falling ...,-0.293,as falling oil price takes toll


In [9]:
pickle.dump(head_data,open(r"D:\PythonCodes\Sentiment-Analysis\Code\head_data_1.dat","wb"),protocol=2)
pickle.dump(post_data,open(r"D:\PythonCodes\Sentiment-Analysis\Code\post_data_1.dat","wb"),protocol=2)


In [10]:
len(head_data),len(post_data)

(93, 99)

In [37]:
from keras.preprocessing.text import Tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


all_data=[]
for i in train_data['sentence']:
    all_data.append(i)
for i in head_data['sentence']:
    all_data.append(i)
for i in post_data['sentence']:
    all_data.append(i)
tokenizer = create_tokenizer(all_data)
vocab_size = len(tokenizer.word_index) + 1

In [38]:
vocab_size

3340

### Generate Word Embedding

In [39]:
import numpy as np
def word2vec(fname, save_file,save_file1):
    sentences = []
    for line in fname:
            sentence = line
            sentences.append(nltk.word_tokenize(sentence))
    #sentences
   # print(sentences[:3])
    model = Word2Vec(sentences, size=300, min_count=1, window=5, sg=1, iter=10)
    embedding_matrix = np.zeros((vocab_size, 300))
    weights = model.wv.syn0
    d = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    embeddings_index = {}
    i=0
    for item in d:
        embeddings_index[item] = weights[d[item], :]
        embedding_matrix[i]=weights[d[item], :]
        i+=1
        #print(embeddings_index)
    pickle.dump(embeddings_index, open(save_file1, 'wb'))
    pickle.dump(embedding_matrix, open(save_file, 'wb'))

    return embeddings_index,embedding_matrix



w2v,embedding= word2vec(train_data['sentence'], 'D:\PythonCodes\Sentiment-Analysis\Code\Other team\embeddings_300_dim.dat','D:\PythonCodes\Sentiment-Analysis\Code\Other team\embedding_index.dat')

C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # This is added back by InteractiveShellApp.init_path()


In [12]:
embedding=pickle.load(open(r'D:\PythonCodes\Sentiment-Analysis\Code\Other team\embeddings_300_dim.dat','rb'))

In [21]:
len(eval('embedding'))

3340

In [162]:
train_data.head()

,aspect,sentence,sentiment,snippet
0,Rumors,ab inbrev latest bid said unlikely win COMPANY...,0.127,latest bid said unlikely to win sabmiller's ap...
1,Appointment,COMPANY shakes board two new business chiefs t...,-0.074,"shakes up board with two new business chiefs, ..."
2,Appointment,press serco set appoint roy gardner ex COMPANY...,0.195,"set to appoint roy gardner, ex-centrica, as ch..."
3,Appointment,press COMPANY set appoint roy gardner ex centr...,0.122,"set to appoint roy gardner, ex-centrica, as ch..."
4,Strategy,barclays sells benchmark indices unit COMPANY,0.281,sells benchmark indices unit


In [95]:
from sklearn import preprocessing
from keras.utils import to_categorical 

def convert_lables (trainY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,27),le.classes_

In [171]:
import numpy as np
import pandas as pd
import nltk

def isinteger(_str):
    return _str.strip().isdigit()


def isfloat(_str):
    return sum([n.isdigit() for n in _str.strip().split('.')]) == 2


def get_index(_list, item, start_index=0):
    for i in range(start_index, len(_list)):
        if item == _list[i]:
            return i
    raise ("can not find %s in %s" % (item, list))
    
def convert_lables (trainY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,27),le.classes_

def read_data_for_aspect(t=train_data,pre_trained_vectors=embedding):

    data_csv = t
    embedding_size=300
    one_aspect=False
    print(type(t))
    stop_words = [',', '.', ':', ';', '?', '(', ')', '[', ']', '!', '@', '#', '%', '$', '*', '-', '/', '&', '``', "''"]

    max_context_len = 0
    word2idx = {}    # Index 0 represents words we haven't met before
    context = []
    context_len = []
    aspect_class = []

    for index, row in data_csv.iterrows():
        word_list = nltk.word_tokenize(row['sentence'].strip())

        context_words = [word for word in word_list
                         if word not in stop_words and isinteger(word) is False and isfloat(word) is False
                         and word != 'COMPANY']
        #print("Context",context_words)
        words_have_vector = [word for word in context_words if word in pre_trained_vectors]
        #print("VECTOR",words_have_vector)

        # make sure most words can find their embedding vectors
        if len(words_have_vector) / float(len(context_words)) < 0.8:
            continue

        max_context_len = max(max_context_len, len(words_have_vector))

        idx = []
        for word in words_have_vector:
            if word not in word2idx:
                word2idx[word] = len(word2idx)+1    # Index 0 represents absent words, so start from 1
            idx.append(word2idx[word])

        context.append(idx)
        context_len.append(len(words_have_vector))

        aspect_class.append(row['aspect'])

    # convert to numpy format
    context_npy = np.zeros(shape=[len(context), max_context_len])
    for i in range(len(context)):
        context_npy[i, :len(context[i])] = context[i]

    aspect_class_npy, onehot_mapping = convert_lables(aspect_class)

    train_d = list()
    train_d.append(context_npy)          # [data_size, max_context_len]
    train_d.append(np.array(context_len))    # [data_size,]
    train_d.append(aspect_class_npy)     # [data_size, aspect_class]

    word_embeddings = np.zeros([len(word2idx)+1, embedding_size])
    for word in word2idx.keys():
        word_embeddings[word2idx[word]] = pre_trained_vectors[word]

    return train_d, word_embeddings, word2idx, max_context_len, onehot_mapping


def read_data_for_senti(t=train_data,pre_trained_vectors=embedding):
    embedding_size=300
    one_aspect=False

    #data_csv = pd.read_csv(fname, sep='\t', header=None, index_col=None,
    #                       names=['text', 'aspect_l1', 'aspect_l2', 'score'])
    data_csv=t
    data_csv = data_csv.sample(frac=1).reset_index(drop=True)   # shuffle data

    stop_words = [',', '.', ':', ';', '?', '(', ')', '[', ']', '!', '@', '#', '%', '$', '*', '-', '/', '&', '``', "''"]

    max_context_len = 0
    word2idx = {}    # Index 0 represents words we haven't met before
    aspect2idx = {}
    context = []
    context_len = []
    loc_info = []
    aspect = []
    score = []
    # aspect_class = []

    for index, row in data_csv.iterrows():
        word_list = nltk.word_tokenize(row.sentence.strip())

        context_words = [word for word in word_list
                         if word not in stop_words and isinteger(word) is False and isfloat(word) is False
                         and word != 'COMPANY']

        words_have_vector = [word for word in context_words if word in pre_trained_vectors]

        # make sure most words can find their embedding vectors
        if len(words_have_vector) / float(len(context_words)) < 0.8:
            continue

        #max_context_len = max(max_context_len, len(words_have_vector))
        max_context_len=18
        idx, distance = [], []
        start_index = 0
        stock_loc = get_index(word_list, 'COMPANY')
        
            
        for word in words_have_vector:
            if word not in word2idx:
                word2idx[word] = len(word2idx)+1    # Index 0 represents absent words, so start from 1
            idx.append(word2idx[word])
            word_loc = get_index(word_list, word, start_index=start_index)
            start_index = word_loc + 1
            distance.append(1 - abs(word_loc-stock_loc) / len(word_list))
        context.append(idx)
        context_len.append(len(words_have_vector))
        loc_info.append(distance)

        if one_aspect is True:      # consider there is only one abstract aspect
            aspect.append(0)
        else:
            if row.aspect not in aspect2idx:
                aspect2idx[row.aspect] = len(aspect2idx)
            aspect.append(aspect2idx[row.aspect])

        score.append([row.sentiment])

    if one_aspect is True:
        aspect2idx['one_aspect'] = len(aspect2idx)

    # convert to numpy format
    context_npy = np.zeros(shape=[len(context), max_context_len])
    loc_info_npy = np.zeros(shape=[len(loc_info), max_context_len])

    for i in range(len(context)):
        context_npy[i, :len(context[i])] = context[i]
        loc_info_npy[i, :len(loc_info[i])] = loc_info[i]

    data = list()
    data.append(context_npy)          # [data_size, max_context_len]
    data.append(np.array(context_len))    # [data_size,]
    data.append(loc_info_npy)         # [data_size, max_context_len]
    data.append(np.array(aspect))     # [data_size]
    data.append(np.array(score))      # [data_size, 1]

    word_embeddings = np.zeros([len(word2idx)+1, embedding_size])
    for word in word2idx.keys():
        word_embeddings[word2idx[word]] = pre_trained_vectors[word]

    return data, word_embeddings, word2idx, aspect2idx, max_context_len

In [174]:
import argparse
    
def _clear_flags():
    """Resets Tensorflow's FLAG values"""
    #pylint: disable=W0212
    for flag_key in dir(tf.app.flags.FLAGS):
        delattr(tf.app.flags.FLAGS, flag_key)
    #tf.app.flags.FLAGS = tf.app.flags._FlagValues()
    tf.app.flags._global_parser = argparse.ArgumentParser()
    

In [175]:
#train_data.head()

In [234]:

import os
import sys
import logging
import numpy as np
import pickle
import tensorflow as tf
#import utils
#from read_data import read_data_for_senti
#from model import DeepMem
#from model import AT_LSTM
_clear_flags()

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

# common hyper-parameter
flags.DEFINE_integer('n_word', 0, 'number of words')
flags.DEFINE_integer('n_aspect', 27, 'number of aspects')
flags.DEFINE_integer('max_len', 0, 'max length of one sentence')
flags.DEFINE_integer('embedding_dim', 300, 'word embedding dimension')
flags.DEFINE_integer('n_epoch', 50, 'max epoch to train')
flags.DEFINE_integer('batch_size', 64, 'batch size')
flags.DEFINE_integer('early_stopping_step', 3, "if loss doesn't descend in 3 epochs, stop training")
flags.DEFINE_integer('train_time', 0, 'train time')
flags.DEFINE_float('stddev', 0.01, 'weight initialization stddev')
flags.DEFINE_float('l2_reg', 0.001, 'l2 regularization')
flags.DEFINE_boolean('show', True, 'print train progress')
flags.DEFINE_boolean('embed_trainable', True, 'whether word embeddings are trainable')
flags.DEFINE_boolean('one_aspect', False,
                     'whether consider all texts are related to one aspect (in an abstract level)')

# hyper-parameter for deep memory model
flags.DEFINE_integer('n_hop', 5, 'number of hops')
flags.DEFINE_boolean('use_loc_info', True, 'whether to add location attention')
flags.DEFINE_string('classifier_type', 'lstm', 'type of classification model: lstm, cnn')
# hyper-parameter for attention-based & target-dependent lstm model
flags.DEFINE_integer('hidden_size', 300, "lstm's hidden units")
flags.DEFINE_integer('n_layer', 3, 'number of lstm layers')
flags.DEFINE_boolean('is_multi', True, 'whether to use multi lstm')
flags.DEFINE_string('lstm_type', 'at', 'type of lstm model')
#flags.DEFINE_string('model_path', '.', 'path to save model')
#flags.DEFINE_string('model_name', 'm', 'model_name')

#logging.basicConfig(level=logging.DEBUG,
#                    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
#                    datefmt='%a, %d %b %Y %H:%M:%S', filename='./log/train.log', filemode='a')

def split(data, i , fold_size):
        current = i
        size = fold_size
        
        index = np.arange(len(data[0]))
        lower_bound = index >= current*size
        upper_bound = index < (current + 1)*size
        v_region = lower_bound*upper_bound
        
        train_data=list()
        v_data=list()
        for j in range (len(data)):
            v_data.append(data[j][v_region])
            train_data.append(data[j][~v_region])
        
        
        return train_data, v_data
    
def train_model(train,valid, word_embeddings,model_type=""):
    #n_fold = 10
    #fold_size = int(len(data[0]) / n_fold)
    #loss_list, acc_list = [], []
    
    best_acc=0
    graph = tf.Graph()
    with tf.Session(graph=graph) as sess:
        model = Classifier(FLAGS, sess)
        model.build_model()
        _, _ = model.run(train, valid, word_embeddings) 
       
     
    
    return models
        
    #logging.debug(model_type + ' 10fold_mse: ' + str(avg_mse) + '\t10fold_r2 :' + str(avg_r2))
    #logging.debug(model_type + ' 10fold_mse_std: ' + str(np.std(mse_list)) + '\t10fold_r2_std :' + str(np.std(r2_list)))
    

def main():
    pre_trained_vectors = w2v
    data, word_embeddings, word2idx, max_context_len, onehot_mapping = read_data_for_aspect(train_data,pre_trained_vectors)
    head_data,_,_,_,_ = read_data_for_aspect(train_data,pre_trained_vectors)   
    post_data,_,_,_,_ = read_data_for_aspect(train_data,pre_trained_vectors)   

    #print(data)
    FLAGS.max_len = max_context_len
    FLAGS.n_word = word_embeddings.shape[0]


    print('unique words embedding: ', word_embeddings.shape)
    print('max sentence len: ', max_context_len)

    model = train_model(data,post_data, word_embeddings)
    
    return model


  

In [235]:
#len(train_data)

In [236]:
#validation_data.head()

In [244]:
import sklearn as sk
class Classifier(object):
    def __init__(self, config, sess):
        self.edim = config.embedding_dim
        self.n_epoch = config.n_epoch
        self.batch_size = config.batch_size
        self.n_class = 27
        self.n_layer = config.n_layer
        self.early_stopping_step = config.early_stopping_step
        self.stddev = config.stddev
        self.hidden_size = config.hidden_size
        self.n_word = config.n_word
        self.max_len = config.max_len
        self.classifier_type = config.classifier_type
        self.l2_reg = config.l2_reg
        self.show = config.show
        self.embed_trainable = config.embed_trainable
        self.is_multi = config.is_multi
    

        self.sess = sess

        # input
        self.context = tf.placeholder(shape=[None, self.max_len], dtype=tf.int32, name='context')
        self.aspect_class = tf.placeholder(shape=[None, self.n_class], dtype=tf.int32, name='aspect_class')
        self.context_len = tf.placeholder(dtype=tf.int32, name='context_len')
        self.keep_prob = tf.placeholder(dtype=tf.float32)

        # word embeddings will be optimized if trainable == True
        self.word_embeddings = tf.Variable(tf.random_uniform([self.n_word, self.edim], -1.0, 1.0),
                                           name='word_embeddings', trainable=self.embed_trainable)

        # batch context embedding & aspect embedding
        self.context_embed = tf.nn.embedding_lookup(self.word_embeddings, self.context,
                                                    name='context_embed')  # [batch_size, max_len, edim]

        # Weights & Bias
        self.W_hidden = tf.Variable(tf.truncated_normal([self.hidden_size, 128], stddev=self.stddev), name='W_hidden')
        self.B_hidden = tf.Variable(tf.constant(0.1, shape=[128]), name='B_hidden')

        self.W_final = tf.Variable(tf.truncated_normal([128, self.n_class], stddev=self.stddev), name='W_final')

        self.B_final = tf.Variable(tf.constant(0.1, shape=[self.n_class]), name='B_final')

        self.predict = None
        self.loss = None
        self.accuracy = None
        self.train_step = None
        self.correct_prediction=None
        self.best_loss = 128.0
        self.best_epoch = 0
        self.best_accuracy = 0.0
        self.stopping_step = 0

        
    def multi_lstm(self):
        batch_size = tf.shape(self.context_embed)[0]
        if self.is_multi:
            stacked_lstm = []
            for i in range(self.n_layer):
                stacked_lstm.append(tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(self.hidden_size,
                                                                                          state_is_tuple=True),
                                                                  output_keep_prob=self.keep_prob))
            multi_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_lstm, state_is_tuple=True)
        else:
            multi_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(self.hidden_size,
                                                                                    state_is_tuple=True),
                                                            output_keep_prob=self.keep_prob)
        initial_state = multi_lstm_cell.zero_state(batch_size, dtype=tf.float32)
        return multi_lstm_cell, initial_state

    def dynamic_rnn(self, lstm_cell, inputs, context_len, initial_state, max_len, out_type='all'):
        outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                           inputs=inputs,
                                           sequence_length=context_len,
                                           initial_state=initial_state,
                                           dtype=tf.float32)  # [batch_size, max_len, hidden_size]
        batch_size = tf.shape(outputs)[0]
        if out_type == 'last':
            indexs = tf.range(0, batch_size) * max_len + (context_len - 1)
            outputs = tf.gather(tf.reshape(outputs, [-1, self.hidden_size]), indexs)  # [batch_size, hidden_size]
        elif outputs == 'all_avg':
            outputs = AT_LSTM.reduce_mean(outputs, context_len)  # [batch_size, hidden_size]
        return outputs

    def lstm_model(self):
        multi_lstm_cell, initial_state = self.multi_lstm()
        lstm_output = self.dynamic_rnn(multi_lstm_cell, self.context_embed, self.context_len, initial_state,
                                       self.max_len, out_type='last')  # [batch_size, hidden]
        hidden_output = tf.matmul(lstm_output, self.W_hidden) + self.B_hidden
        predict = tf.nn.softmax(tf.matmul(hidden_output, self.W_final) + self.B_final)
        return predict

    def cnn_model(self):
        conv_filter_w = tf.Variable(tf.truncated_normal([3, self.edim, 64],
                                                        stddev=self.stddev), name='W_hidden')
        conv_filter_b = tf.Variable(tf.constant(0.1, shape=[64]))
        conv_output = tf.nn.conv1d(self.context_embed, conv_filter_w, stride=2, padding='SAME')
        relu_output = tf.nn.relu(conv_output + conv_filter_b)

        batch_size = tf.shape(relu_output)[0]
        flatten = tf.reshape(relu_output, [batch_size, -1])

        W_hidden = tf.Variable(tf.truncated_normal([14 * 64, 128], stddev=self.stddev), name='W_hidden_cnn')
        B_hidden = tf.Variable(tf.constant(0.1, shape=[128]), name='B_hidden_cnn')
        hidden_output = tf.matmul(flatten, W_hidden) + B_hidden
        predict = tf.nn.softmax(tf.matmul(hidden_output, self.W_final) + self.B_final)
        return predict

    def build_model(self):
        if self.classifier_type == 'lstm':
            self.predict = self.lstm_model()
        elif self.classifier_type == 'cnn':
            self.predict = self.cnn_model()
        else:
            self.predict = self.lstm_model()
        self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=self.aspect_class, logits=self.predict))
        self.correct_prediction = tf.equal(tf.argmax(self.predict, 1), tf.argmax(self.aspect_class, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        self.train_step = tf.train.AdamOptimizer().minimize(self.loss)

    def train(self, train_data, valid_data, word_embeddings):
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        self.sess.run(self.word_embeddings.assign(word_embeddings))

        context, context_len, aspect_class = train_data
        loops = int(len(context) / 64)
        for epoch in range(self.n_epoch):
            avg_loss, avg_accuracy = 0.0, 0.0
            for i in range(loops):
                feed_dict = {self.context: context[i * self.batch_size: (i + 1) * self.batch_size],
                             self.context_len: context_len[i * self.batch_size: (i + 1) * self.batch_size],
                             self.aspect_class: aspect_class[i * self.batch_size: (i + 1) * self.batch_size],
                             self.keep_prob: 0.5}
                #_, loss, accuracy = self.sess.run([self.train_step, self.loss, self.accuracy], feed_dict=feed_dict)
                self.correct_prediction = tf.argmax(self.predict, 1)
                _,y_pred,val_accuracy  = self.sess.run([self.train_step, self.accuracy, self.correct_prediction], feed_dict=feed_dict)
                print ("validation accuracy:", val_accuracy)
                y_true = np.argmax(self.aspect_class,1)
                print ("f1_score", sk.metrics.f1_score(y_true, y_pred,average='macro'))
                print ("f1_score Weighted", sk.metrics.f1_score(y_true, y_pred,average='weighted'))

                avg_loss += loss
                avg_accuracy += accuracy

            avg_loss /= loops
            avg_accuracy /= loops

            
            if self.show:
                print(self.classifier_type, ' epoch:', epoch, 'train_loss:', avg_loss, 'train_accuracy:', avg_accuracy)

            #saver.save(self.sess, os.path.join(self.save_model_path + self.save_model_name), global_step=epoch)

            valid_loss, valid_accuracy = self.valid(valid_data)

            if valid_accuracy > self.best_accuracy:
                self.best_loss = valid_loss
                self.best_accuracy = valid_accuracy
                self.best_epoch = epoch
                self.stopping_step = 0
            else:
                self.stopping_step += 1
            if self.stopping_step >= self.early_stopping_step:
                #logging.debug(self.classifier_type + ' early stopping is trigger at epoch: ' + str(epoch))
                if self.show:
                    print(self.classifier_type, 'early stopping is trigger at epoch: ', epoch)
                break

        if self.show:
            print(self.classifier_type, 'best epoch:', self.best_epoch, 'best loss:', self.best_loss,
                  'best accuracy:', self.best_accuracy)
    
        return self.best_loss, self.best_accuracy

    def valid(self, valid_data):
        context, context_len, aspect_class = valid_data
        feed_dict = {self.context: context,
                     self.context_len: context_len,
                     self.aspect_class: aspect_class,
                     self.keep_prob: 1.0}
        loss, accuracy = self.sess.run([self.loss, self.accuracy], feed_dict=feed_dict)
        if self.show:
            print(self.classifier_type, ' valid_loss:', loss, 'valid_accuracy:', accuracy)
        return loss, accuracy

    def run(self, train_data, valid_data, word_embeddings):
        loss, accuracy = self.train(train_data, valid_data, word_embeddings)
        return loss, accuracy
    
    

In [245]:
model1=main()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
unique words embedding:  (3048, 300)
max sentence len:  18


C:\Users\simcy\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


validation accuracy: [26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26
 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26
 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26 26]


AxisError: axis 1 is out of bounds for array of dimension 1

In [223]:
# attention-based lstm
class AT_LSTM(object):
    def __init__(self, config, sess):
        self.edim = config.embedding_dim
        self.n_epoch = config.n_epoch
        self.batch_size = config.batch_size
        self.n_aspect = config.n_aspect
        self.n_layer = config.n_layer
        self.early_stopping_step = config.early_stopping_step
        self.stddev = config.stddev
        self.hidden_size = config.hidden_size
        self.n_word = config.n_word
        self.max_len = config.max_len
        self.lstm_type = config.lstm_type
        self.l2_reg = config.l2_reg
        self.show = config.show
        self.embed_trainable = config.embed_trainable
        self.is_multi = config.is_multi
        #self.save_model_path = config.model_path
        #self.save_model_name = config.model_name
        self.train_time = config.train_time

        self.sess = sess

        # input
        self.context = tf.placeholder(shape=[None, self.max_len], dtype=tf.int32, name='context')
        self.aspect = tf.placeholder(dtype=tf.int32, name='aspect')
        self.score = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='score')
        self.context_len = tf.placeholder(dtype=tf.int32, name='context_len')
        self.keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')

        # aspect embeddings will be optimized along with training process
        # word embeddings will be optimized if trainable == True
        self.word_embeddings = tf.Variable(tf.random_uniform([self.n_word, self.edim], -1.0, 1.0),
                                           name='word_embeddings', trainable=self.embed_trainable)
        self.aspect_embeddings = tf.Variable(tf.random_uniform([self.n_aspect, self.edim], -1.0, 1.0),
                                             name='aspect_embeddings')

        # batch context embedding & aspect embedding
        self.context_embed = tf.nn.embedding_lookup(self.word_embeddings, self.context,
                                                    name='context_embed')  # [batch_size, max_len, edim]
        self.aspect_embed = tf.nn.embedding_lookup(self.aspect_embeddings, self.aspect,
                                                   name='aspect_embed')  # [batch_size, edim]

        # Weights
        self.W_concat = tf.Variable(tf.truncated_normal([self.edim + self.hidden_size, self.edim + self.hidden_size],
                                                        stddev=self.stddev), name='W_concat')
        self.W_softmax = tf.Variable(tf.truncated_normal([self.edim + self.hidden_size, 1], stddev=self.stddev),
                                     name='W_softmax')
        self.Wp = tf.Variable(tf.truncated_normal([self.hidden_size, self.hidden_size], stddev=self.stddev),
                              name='Wp')
        self.Wx = tf.Variable(tf.truncated_normal([self.hidden_size, self.hidden_size], stddev=self.stddev),
                              name='Wx')
        self.W_final = tf.Variable(tf.truncated_normal([self.hidden_size, 1], stddev=self.stddev), name='W_final')

        self.predict = None
        self.mse = None
        self.r2 = None
        self.train_step = None

        self.best_mse = 4.0
        self.best_epoch = 0
        self.best_r2 = 0.0
        self.stopping_step = 0

       # logging.basicConfig(level=logging.DEBUG,
       #                     format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
       #                     datefmt='%a, %d %b %Y %H:%M:%S', filename='./data/AT_LSTM_watch.log', filemode='a')

    def multi_lstm(self):
        batch_size = tf.shape(self.context_embed)[0]

        if self.is_multi:
            stacked_lstm = []
            for i in range(self.n_layer):
                stacked_lstm.append(tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(self.hidden_size,
                                                                                          state_is_tuple=True),
                                                                  output_keep_prob=self.keep_prob))
            multi_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_lstm, state_is_tuple=True)
        else:
            multi_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(self.hidden_size,
                                                                                    state_is_tuple=True),
                                                            output_keep_prob=self.keep_prob)

        initial_state = multi_lstm_cell.zero_state(batch_size, dtype=tf.float32)

        return multi_lstm_cell, initial_state

    def dynamic_rnn(self, lstm_cell, inputs, context_len, initial_state, max_len, out_type='all'):
        outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                           inputs=inputs,
                                           sequence_length=context_len,
                                           initial_state=initial_state,
                                           dtype=tf.float32)  # [batch_size, max_len, hidden_size]

        batch_size = tf.shape(outputs)[0]
        if out_type == 'last':
            indexs = tf.range(0, batch_size) * max_len + (context_len - 1)
            outputs = tf.gather(tf.reshape(outputs, [-1, self.hidden_size]), indexs)  # [batch_size, hidden_size]
        elif outputs == 'all_avg':
            outputs = AT_LSTM.reduce_mean(outputs, context_len)  # [batch_size, hidden_size]
        return outputs

    def cnn_model(self):
        conv_filter_w = tf.Variable(tf.truncated_normal([3, self.edim, 64],
                                                        stddev=self.stddev), name='W_hidden')
        conv_filter_b = tf.Variable(tf.constant(0.1, shape=[64]))
        conv_output = tf.nn.conv1d(self.context_embed, conv_filter_w, stride=1, padding='SAME')
        relu_output = tf.nn.relu(conv_output + conv_filter_b)

        relu_output = tf.expand_dims(relu_output, 1)
        max_pool = tf.nn.max_pool(relu_output, ksize=[1, 1, 2, 1], strides=[1, 1, 2, 1], padding='SAME')

        batch_size = tf.shape(max_pool)[0]
        flatten = tf.reshape(max_pool, [batch_size, -1])

        W_hidden = tf.Variable(tf.truncated_normal([9 * 64, 128], stddev=self.stddev), name='W_hidden_cnn')
        B_hidden = tf.Variable(tf.constant(0.1, shape=[128]), name='B_hidden_cnn')
        hidden_output = tf.matmul(flatten, W_hidden) + B_hidden

        W_final = tf.Variable(tf.truncated_normal([128, 1], stddev=self.stddev), name='W_final')
        predict = tf.matmul(hidden_output, W_final, name='predict')
        return predict

    # orignal lstm model
    def original_model(self):
        multi_lstm_cell, initial_state = self.multi_lstm()
        lstm_output = self.dynamic_rnn(multi_lstm_cell, self.context_embed, self.context_len, initial_state,
                                       self.max_len, out_type='last')  # [batch_size, hidden]

        predict = tf.matmul(lstm_output, self.W_final, name='predict')

        return predict

    # at_model: use attention mechanism
    # concatenate the aspect vector into the sentence hidden representations for computing attention weights
    def at_model(self):
        batch_size = tf.shape(self.context_embed)[0]
        aspect_expand = tf.tile(self.aspect_embed, [1, self.max_len])
        aspect_expand = tf.reshape(aspect_expand, [-1, self.max_len, self.edim],
                                   name='aspect_expand')  # [batch_size, max_len, edim]

        multi_lstm_cell, initial_state = self.multi_lstm()
        lstm_output = self.dynamic_rnn(multi_lstm_cell, self.context_embed, self.context_len, initial_state,
                                       self.max_len, out_type='all')  # [batch_size, max_len, hidden]

        w_concat_tile = tf.reshape(tf.tile(self.W_concat, [batch_size, 1]),
                                   [batch_size, self.edim + self.hidden_size, -1],
                                   name='w_concat_title')  # [batch_size, edim+hidden_size, edim+hidden_size]
        M = tf.nn.tanh(tf.matmul(tf.concat([lstm_output, aspect_expand], axis=2), w_concat_tile),
                       name='M')  # [batch_size, max_len, edim+hidden_size]
        w_softmax_tile = tf.reshape(tf.tile(self.W_softmax, [batch_size, 1]),
                                    [batch_size, -1, 1])  # [batch_size, edim+hidden_size, 1]
        w_mul_M = tf.reshape(tf.matmul(M, w_softmax_tile), [batch_size, 1, -1],
                             name='w_mul_M')  # [batch_size, 1, max_len]

        alpha = AT_LSTM.softmax(w_mul_M, self.context_len, self.max_len)  # [batch_size, 1, max_len)

        r = tf.reshape(tf.matmul(alpha, lstm_output), [batch_size, self.hidden_size],
                       name='r')  # weighted representation of sentence with given aspect: [batch_size, hidden_size]

        # get last output of lstm
        index = tf.range(batch_size) * self.max_len + (self.context_len - 1)
        lstm_last = tf.gather(tf.reshape(lstm_output, [-1, self.hidden_size]), index)  # [batch_size, hidden_size]

        h = tf.tanh(tf.matmul(r, self.Wp) + tf.matmul(lstm_last, self.Wx))  # final sentence representation

        predict = tf.matmul(h, self.W_final, name='predict')

        return predict

    # ae_model: additionally append the aspect vector into the input word vectors
    def ae_model(self):
        aspect_expand = tf.tile(self.aspect_embed, [1, self.max_len])
        aspect_expand = tf.reshape(aspect_expand, [-1, self.max_len, self.edim],
                                   name='aspect_expand')  # [batch_size, max_len, edim]
        inputs = tf.concat([self.context_embed, aspect_expand], axis=2, name='inputs')

        multi_lstm_cell, initial_state = self.multi_lstm()
        lstm_output = self.dynamic_rnn(multi_lstm_cell, inputs, self.context_len, initial_state, self.max_len,
                                       out_type='last')  # [batch_size, hidden]

        predict = tf.matmul(lstm_output, self.W_final, name='predict')

        return predict

    # atae_model: attention mechanism & aspect vector as input
    def atae_model(self):
        batch_size = tf.shape(self.context_embed)[0]
        aspect_expand = tf.tile(self.aspect_embed, [1, self.max_len])
        aspect_expand = tf.reshape(aspect_expand, [-1, self.max_len, self.edim],
                                   name='aspect_expand')  # [batch_size, max_len, edim]
        inputs = tf.concat([self.context_embed, aspect_expand], axis=2, name='inputs')

        multi_lstm_cell, initial_state = self.multi_lstm()
        lstm_output = self.dynamic_rnn(multi_lstm_cell, inputs, self.context_len, initial_state, self.max_len,
                                       out_type='all')  # [batch_size, max_len, hidden]

        w_concat_tile = tf.reshape(tf.tile(self.W_concat, [batch_size, 1]),
                                   [batch_size, self.edim + self.hidden_size, -1],
                                   name='w_concat_title')  # [batch_size, edim+hidden_size, edim+hidden_size]
        M = tf.nn.tanh(tf.matmul(tf.concat([lstm_output, aspect_expand], axis=2), w_concat_tile),
                       name='M')  # [batch_size, max_len, edim+hidden_size]
        w_softmax_tile = tf.reshape(tf.tile(self.W_softmax, [batch_size, 1]),
                                    [batch_size, -1, 1])  # [batch_size, edim+hidden_size, 1]
        w_mul_M = tf.reshape(tf.matmul(M, w_softmax_tile), [batch_size, 1, -1],
                             name='w_mul_M')  # [batch_size, 1, max_len]

        alpha = AT_LSTM.softmax(w_mul_M, self.context_len, self.max_len)  # [batch_size, 1, max_len)

        r = tf.reshape(tf.matmul(alpha, lstm_output), [batch_size, self.hidden_size],
                       name='r')  # weighted representation of sentence with given aspect: [batch_size, hidden_size]

        # get last output of lstm
        index = tf.range(batch_size) * self.max_len + (self.context_len - 1)
        lstm_last = tf.gather(tf.reshape(lstm_output, [-1, self.hidden_size]), index)  # [batch_size, hidden_size]

        h = tf.tanh(tf.matmul(r, self.Wp) + tf.matmul(lstm_last, self.Wx))  # final sentence representation

        predict = tf.matmul(h, self.W_final, name='predict')

        return predict

    def build_model(self):
        if self.lstm_type == 'at':
            self.predict = self.at_model()
        elif self.lstm_type == 'ae':
            self.predict = self.ae_model()
        elif self.lstm_type == 'atae':
            self.predict = self.atae_model()
        elif self.lstm_type == 'cnn':
            self.predict = self.cnn_model()
        else:
            self.predict = self.original_model()

        reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        self.mse = tf.reduce_mean(tf.square(self.score - self.predict)) + sum(reg_loss)

        score_mean = tf.reduce_mean(self.score)
        self.r2 = 1 - tf.reduce_sum(tf.square(self.score - self.predict)) / tf.reduce_sum(
            tf.square(self.score - score_mean))

        self.train_step = tf.train.AdamOptimizer().minimize(self.mse)

    def train(self, train_data, valid_data,head_data,post_data,word_embeddings, ):
        self.sess.run(tf.global_variables_initializer())
        self.sess.run(tf.local_variables_initializer())
        self.sess.run(self.word_embeddings.assign(word_embeddings))
      #  saver = tf.train.Saver(max_to_keep=self.n_epoch)

        context, context_len, loc_info, aspect, score = train_data
        print(len(context))
        loops = int(len(context) / self.batch_size)
        print("Loops ",loops)
        for epoch in range(self.n_epoch):
            avg_mse, avg_r2 = 0.0, 0.0
            for i in range(loops):
                feed_dict = {self.context: context[i * self.batch_size: (i + 1) * self.batch_size],
                             self.context_len: context_len[i * self.batch_size: (i + 1) * self.batch_size],
                             self.aspect: aspect[i * self.batch_size: (i + 1) * self.batch_size],
                             self.score: score[i * self.batch_size: (i + 1) * self.batch_size],
                             self.keep_prob: 0.5}
                # print(self.sess.run(self.predict, feed_dict=feed_dict))
                _, mse, r2 = self.sess.run([self.train_step, self.mse, self.r2], feed_dict=feed_dict)

                avg_mse += mse
                avg_r2 += r2

            avg_mse /= loops
            avg_r2 /= loops
            #logging.debug(self.lstm_type + ' ' + str(epoch) + ' train_mse: ' +
            #              str(avg_mse) + '\ttrain_r2 :' + str(avg_r2))

            if self.show:
                print(self.lstm_type, ' epoch:', epoch, 'train_mse:', avg_mse, 'train_r2:', avg_r2)

            #saver.save(self.sess, os.path.join(self.save_model_path, self.save_model_name), global_step=epoch)
            
            valid_mse, valid_r2 = self.valid(valid_data)

            if valid_mse < self.best_mse:
                self.best_mse = valid_mse
                self.best_r2 = valid_r2
                self.best_epoch = epoch
                self.stopping_step = 0
            else:
                self.stopping_step += 1
            if self.stopping_step >= self.early_stopping_step:
                #logging.debug(self.lstm_type + ' early stopping is trigger at epoch: ' + str(epoch))
                if self.show:
                    print(self.lstm_type + ' early stopping is trigger at epoch: ', epoch)
                break

        if self.show:
            print(self.lstm_type + ' best epoch:', self.best_epoch, 'best mse:', self.best_mse, 'best r2:', self.best_r2)
       # logging.debug(self.lstm_type + ' best epoch: ' + str(self.best_epoch) +
       #               '\tbest mse: ' + str(self.best_mse) + '\tbest r2 :' + str(self.best_r2))
        
        #only_save_best_epoch(self.save_model_path, self.save_model_name, self.best_epoch, self.train_time)
        
        head_mse, head_r2 = self.valid(head_data)
        post_mse, post_r2 = self.valid(post_data)
        
        print("Headline")
        print("MSE : ",head_mse)
        print("R2 : ",post_r2)
        
        print("Post")
        print("MSE : ",post_mse)
        print("R2 : ",post_r2)
        
        return self.best_mse, self.best_r2

    def valid(self, valid_data):
        context, context_len, loc_info, aspect, score = valid_data

        feed_dict = {self.context: context,
                     self.context_len: context_len,
                     self.aspect: aspect,
                     self.score: score,
                     self.keep_prob: 1.0}

        mse, r2 = self.sess.run([self.mse, self.r2], feed_dict=feed_dict)

       # logging.debug(self.lstm_type + ' valid_mse: ' + str(mse) + '\tvalid_r2 :' + str(r2))

        if self.show:
            print(self.lstm_type, ' valid_mse:', mse, 'valid_r2:', r2)

        return mse, r2

    
    def run(self, train_data, valid_data, head, post ,word_embeddings):
        mse, r2 = self.train(train_data, valid_data,head,post, word_embeddings)
        return mse, r2
    

    @staticmethod
    def reduce_mean(inputs, length):
        """
        :param inputs: 3D tensor->[batch_size, sequence_len, embedding_size]
        :param length: 1D tensor->[batch_size], represent every sample's len, not bigger than sequence_len
        :return: 2D tensor->[batch_size, embedding_size]
        """
        length = tf.reshape(length, [-1, 1])
        outputs = tf.reduce_sum(inputs, axis=1) / length
        return outputs

    @staticmethod
    def softmax(inputs, length, max_length):
        """
        :param inputs: 3D tensor->[batch_size, 1, n_class]
        :param length: 1D tensor->[batch_size], represent every sample's len, not bigger than sequence_len
        :param max_length:
        :return: 3D tensor->[batch_size, 1, n_class]
        """
        length = tf.reshape(length, [-1])
        inputs = tf.exp(inputs)
        mask = tf.reshape(tf.cast(tf.sequence_mask(length, max_length), tf.float32), tf.shape(inputs))
        inputs *= mask
        _sum = tf.reduce_sum(inputs, reduction_indices=2, keepdims=True)
        return inputs / _sum
  

In [160]:
#def only_save_best_epoch(save_model_path,save_model_name,best_epoch,train_time):
    